## Dart Lab Tutorial

#### Starting interactive cells
We will use the cell below as a header file essentially.  We will import all of the nessecary packages from this cell, and use them throughout the rest of the document. Throughout this document, we will be using code from previous cells, and gradually building off of it to add more features. 

In [141]:
import math
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
from pylab import *
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, Math, Latex
from scipy.integrate import quad
from scipy import integrate
from sympy import *
import matplotlib.ticker as plticker

PI = np.pi

### Example: Estimating the Temperature Outside pt. 1
The cell below is going to be our first interactive cell.  In this cell, we are able to adjust the silders to change our temperature value, and also change the size of our icon.  The purpose of this is to introduce the idea of **interactive cells**, and show some of the basic features that they include.

In [142]:
def f(x, y, r):
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('An observation has a value (*),')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    plt.scatter([x], [y], marker='*', s=r, c='r')
    plt.show()

interact(f, x=(-4.0, 4.0), y=fixed(0), r=(10,1000))



interactive(children=(FloatSlider(value=0.0, description='x', max=4.0, min=-4.0), IntSlider(value=505, descrip…

<function __main__.f(x, y, r)>

### Estimating the Temperature outside pt. 2
Here in the cell below, we have a similar graph as before, but added an error distribution (blue curve) that is associated with the instrument.   This **gaussian function** can be used as a basis for future graphs, and will be called upon multiple times in the rest of this document. Instrument builder says thermometer is unbiased with =/- 0.8°C gaussian error.

The red plot is $P(\frac{T}{T_0})$; probability of temperature *given* that $T_0$ was observed.

In [143]:
# Base gaussian function
def gaussian(mu, sigma):
    x=np.linspace(-10, 10, num=1000)
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    plt.plot(x,y, c='r')
    plt.scatter(mu, 0, c='r')
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('An observation has a value (*),')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    plt.show()

interact(gaussian, mu=(-4.0, 4.0), sigma=(0.1, 10))
    

interactive(children=(FloatSlider(value=0.0, description='mu', max=4.0, min=-4.0), FloatSlider(value=5.05, des…

<function __main__.gaussian(mu, sigma)>

### We also have a prior estimate of temperature.
**Any future lines using the color green will be refering to prior PDF.**

In [144]:
# Base gaussian function
def gaussian(mu, sigma):
    x=np.linspace(-10, 10, num=1000)
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    plt.plot(x,y, c='g')
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate of Temperature')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    plt.show()

interact(gaussian, mu=(-4.0, 4.0), sigma=(0.1, 10))
    

interactive(children=(FloatSlider(value=0.0, description='mu', max=4.0, min=-4.0), FloatSlider(value=5.05, des…

<function __main__.gaussian(mu, sigma)>

The green curve is $P(\frac{T}{T_0})$;
probability of temperature given all available **prior** information $C$.

Prior information $C$ can include:

1. Observations of things besides $T$;
2. Model forecast made using observations at earlier times;
3. *A priori* physical constraints $(T > -273.15°C)$;
4. Climatological constraints $(-30°C < T < 40°C)$.

### Combining the Prior Estimate and Observation
#### Bayes Theorem: 
$$P(T|T_0,C)=\frac{P(T_0|T,C)P(T|C)}{P(T_0|C)}$$

The LHS represents the **Posterior**: Probability of $T$ given observations and Prior.  Also called **update** or **analysis**.

While the RHS represents the **likelihood**: Probability that $T_0$ is observed if $T$ is a true value and given prior information $C$.

Rewrite Bayes as:

$$\frac{P(T_0|T,C)P(T|C)}{P(T_0|C)}$$

$$\frac{P(T_0|T,C)P(T|C)}{\int P(T_0|x)P(x|C)dx}$$

$$\frac{\color{red}{P(T_0|T,C)}\color{green}{P(T|C)}}{normalization}$$

Denominator normalizes so Posterior is PDF.

In [145]:
# Base gaussian function
def gaussian(mu, sigma):
    x=np.linspace(-10, 10, num=1000)
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    plt.plot(x,y, c='g')
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate of Temperature')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    plt.text(-3.5, .25, r'Prior PDF')
    plt.show()

interact(gaussian, mu=(-4.0, 4.0), sigma=(0.1, 10))
    

interactive(children=(FloatSlider(value=0.0, description='mu', max=4.0, min=-4.0), FloatSlider(value=5.05, des…

<function __main__.gaussian(mu, sigma)>

In [146]:
def gaussian(x, mu, sigma):
    x=np.linspace(-10, 10, num=1000)
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    return y
    
def plot2gaussians(mu1, sigma1, mu2, sigma2):
    x=np.linspace(-10, 10, num=1000)
    y1=gaussian(x, mu1, sigma1)
    y2=gaussian(x, mu2, sigma2)
    
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate and Observation')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    
    plt.plot(x, y1, c='g')
    plt.plot(x, y2, c='r')
    
    plt.scatter(mu1, 0)
    plt.scatter(mu2, 0)
    plt.show()
    
interact(plot2gaussians, mu1=(-4.0, 4.0), sigma1=(0.1, 10),\
         mu2=(-4.0, 4.0), sigma2=(0.1, 10) )

interactive(children=(FloatSlider(value=0.0, description='mu1', max=4.0, min=-4.0), FloatSlider(value=5.05, de…

<function __main__.plot2gaussians(mu1, sigma1, mu2, sigma2)>

$$\frac{\color{red}{P(T_0|T,C)}\color{green}{P(T|C)}}{normalization}$$

In [151]:
def gaussian(x, mu, sigma):
    x=np.linspace(-10, 10, num=1000)
    y=1/(sigma*np.sqrt(2*np.pi))*np.exp((-1/2)*((x-mu)/sigma)**2)
    return y
    
def plot3gaussians(mu1, sigma1, mu2, sigma2, mu3, sigma3):
    x=np.linspace(-10, 10, num=1000)
    y1=gaussian(x, mu1, sigma1)
    y2=gaussian(x, mu2, sigma2)
    y3=gaussian(x, mu3, sigma3)
    
    plt.axis([-4, 4, -.1, .2])
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.title('Prior Estimate and Observation')
    plt.yticks([0, 0.2, 0.4, 0.6])
    plt.xticks([-4, -2, 0, 2, 4])
    plt.grid(True)
    
    plt.plot(x, y1, c='g')
    plt.plot(x, y2, c='r')
    plt.plot(x, y3, c='b')
    
    plt.scatter(mu1, 0)
    plt.scatter(mu2, 0)
    plt.scatter(mu3, 0)
    plt.show()
    
interact(plot3gaussians, mu1=(-4.0, 4.0), sigma1=(0.1, 10),\
         mu2=(-4.0, 4.0), sigma2=(0.1, 10), mu3=(-4.0, 4.0),\
         sigma3=(0.1, 10))

interactive(children=(FloatSlider(value=0.0, description='mu1', max=4.0, min=-4.0), FloatSlider(value=5.05, de…

<function __main__.plot3gaussians(mu1, sigma1, mu2, sigma2, mu3, sigma3)>